# Protein Folding Project

In [1]:
from gurobipy import *

## Index sets

In [2]:
acids = set() # aminoacids set. 1~50.

for n in range(1, 51):
    acids.add(n)
    

hydro_acids = set() # hydrophobic acids set.
hydro_acids = {2, 4, 5, 6, 11, 12, 17, 20, 21, 25, 27, 28, 30, 31, 33, 37, 44, 46}

print(f"aminoacids set : {acids}")
print(f"\nhydro_acids set : {hydro_acids}")

aminoacids set : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}

hydro_acids set : {2, 4, 5, 6, 11, 12, 17, 20, 21, 25, 27, 28, 30, 31, 33, 37, 44, 46}


## Subscripts

In [3]:
# 'k' in aminoacids set
# 'i', 'j' in hydrophobic acids set. 

## Create model

In [4]:
m = Model("My Protein")

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-06-07


## Decision variables

In [5]:
# close : i 와 j가 가깝게 배치됨: 1 or 0  2차원 결정변수. (i, j)
# 가깝게 배치할 수 있는 i와 j는 i + 2 <= j && i와 j 사이에는 짝수 개의 산이 존재.

# tuplelist 만들기.
close_list = []
for i in hydro_acids:
    for j in hydro_acids:
        if (j > i+2) and ((i-j-1) % 2 == 0):
            close_list.append((i,j))
close_tplist = tuplelist(close_list)

# 결정변수.
close = m.addVars(close_tplist, vtype=GRB.BINARY, name='close')
fold = m.addVars(acids, vtype=GRB.BINARY, name="fold")

## Constraints

In [6]:
# 1. 가깝게 배치를 할 수 있는 소수성 산(in close_tplist)들은 접기를 해야만 가깝게 배치가 가능하다.
# 2. 접기가 발생하는 산의 인덱스 k == int((i+j) / 2)  (i < k < j)
enable_close = []
disable_close = []
for i,j in close_tplist:
    for k in range(i,j):
        if(k == (i + j - 1) / 2):
            enable_close.append((i,j,k))
        else:
            disable_close.append((i,j,k))
enable_close_tplist = tuplelist(enable_close)
disable_close_tplist = tuplelist(disable_close)

m.addConstrs(close[i, j] <= fold[k] for i,j,k in enable_close_tplist)
m.addConstrs(close[i, j] + fold[k] <= 1 for i,j,k in disable_close_tplist)

{(2, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17, 11): <gurobi.Constr

## Objective function

In [7]:
obj_list = []
for i,j,k in enable_close_tplist:
   obj_list.append((i,j)) 
obj_tplist = tuplelist(obj_list)

m.setObjective(quicksum(close[i,j] for i, j in obj_tplist), GRB.MAXIMIZE)

In [8]:
m.write('result.lp')

m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1265 rows, 125 columns and 2530 nonzeros
Model fingerprint: 0xbbe4c460
Variable types: 0 continuous, 125 integer (125 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 420 rows and 33 columns
Presolve time: 0.01s
Presolved: 845 rows, 92 columns, 1779 nonzeros
Variable types: 0 continuous, 92 integer (92 binary)
Found heuristic solution: objective 8.0000000

Root relaxation: objective 3.200000e+01, 99 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.50000    0   82    8.00000   28.50000   256%    

In [26]:
for i,j,k in enable_close_tplist:
    if (close[i,j].x > 0.5): 
        print(f"Close : {i,j}, Fold : {k}.") 

Close : (2, 5), Fold : 3.
Close : (5, 12), Fold : 8.
Close : (6, 11), Fold : 8.
Close : (12, 17), Fold : 14.
Close : (17, 20), Fold : 18.
Close : (20, 25), Fold : 22.
Close : (25, 28), Fold : 26.
Close : (27, 30), Fold : 28.
Close : (30, 33), Fold : 31.
Close : (37, 46), Fold : 41.
